<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition

In [15]:
import glob
import os
import librosa
import numpy as np

In [25]:
! wget https://www.dropbox.com/s/l5hjh95gabnubn8/archive.zip

--2021-09-07 02:26:40--  https://www.dropbox.com/s/l5hjh95gabnubn8/archive.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/l5hjh95gabnubn8/archive.zip [following]
--2021-09-07 02:26:40--  https://www.dropbox.com/s/raw/l5hjh95gabnubn8/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5b9fc6c504c8d0cfe120264ef5.dl.dropboxusercontent.com/cd/0/inline/BVpQ7pPbyXMSVda8EEwnLlY0M_cU5cQcfFLOylAE8Bs-J2mInbPncuKjcOVTQdBzTZlheNJPJqg86Ut0ueZ_8m2oOdWfx5S3iEUjNMjcyBcWb8FuLX9I0digHcUwOzm1kuI5yBSnsK4FryUUuqoV5Gnv/file# [following]
--2021-09-07 02:26:40--  https://uc5b9fc6c504c8d0cfe120264ef5.dl.dropboxusercontent.com/cd/0/inline/BVpQ7pPbyXMSVda8EEwnLlY0M_cU5cQcfFLOylAE8Bs-J2mInbPncuKjcOVTQdBzTZlheNJPJqg86Ut0ueZ_8m2oOdWfx

In [26]:
!unzip archive.zip

Archive:  archive.zip
   creating: archive/
   creating: archive/Actor_01/
  inflating: archive/Actor_01/03-02-01-01-01-01-01.wav  
  inflating: archive/Actor_01/03-02-01-01-01-02-01.wav  
  inflating: archive/Actor_01/03-02-01-01-02-01-01.wav  
  inflating: archive/Actor_01/03-02-01-01-02-02-01.wav  
  inflating: archive/Actor_01/03-02-02-01-01-01-01.wav  
  inflating: archive/Actor_01/03-02-02-01-01-02-01.wav  
  inflating: archive/Actor_01/03-02-02-01-02-01-01.wav  
  inflating: archive/Actor_01/03-02-02-01-02-02-01.wav  
  inflating: archive/Actor_01/03-02-02-02-01-01-01.wav  
  inflating: archive/Actor_01/03-02-02-02-01-02-01.wav  
  inflating: archive/Actor_01/03-02-02-02-02-01-01.wav  
  inflating: archive/Actor_01/03-02-02-02-02-02-01.wav  
  inflating: archive/Actor_01/03-02-03-01-01-01-01.wav  
  inflating: archive/Actor_01/03-02-03-01-01-02-01.wav  
  inflating: archive/Actor_01/03-02-03-01-02-01-01.wav  
  inflating: archive/Actor_01/03-02-03-01-02-02-01.wav  
  inflating: 

In [27]:
# list all files in the dataset
data_path = 'archive'
files = glob.glob(os.path.join(data_path + '/*/', '*.wav'))
files.sort()

In [28]:
# check by showing the first file
files[0]

'archive/Actor_01/03-02-01-01-01-01-01.wav'

In [29]:
# function to extract feature
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name, sr=None)
    stft = np.abs(librosa.stft(X))
    mfcc = np.mean(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    mfcc_std = np.std(librosa.feature.mfcc(
        y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    chroma_std = np.std(librosa.feature.chroma_stft(
        S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    mel_std = np.std(librosa.feature.melspectrogram(
        X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    contrast_std = np.std(librosa.feature.spectral_contrast(
        S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    tonnetz_std = np.std(librosa.feature.tonnetz(
        y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    return (mfcc, chroma, mel, contrast, tonnetz,
            mfcc_std, chroma_std, mel_std, contrast_std, tonnetz_std)

In [30]:
# create empty list to store features and labels
feat = []
lab = []

In [ ]:
# iterate over all files
for file in files:
    print("processing ...", file)
    feat_i = np.hstack(extract_feature(file))
    lab_i = os.path.basename(file).split('-')[2]
    feat.append(feat_i)
    lab.append(int(lab_i)-1)  # make labels start from 0

processing ... archive/Actor_01/03-02-01-01-01-01-01.wav
processing ... archive/Actor_01/03-02-01-01-01-02-01.wav
processing ... archive/Actor_01/03-02-01-01-02-01-01.wav
processing ... archive/Actor_01/03-02-01-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-01-01-01-01.wav
processing ... archive/Actor_01/03-02-02-01-01-02-01.wav
processing ... archive/Actor_01/03-02-02-01-02-01-01.wav
processing ... archive/Actor_01/03-02-02-01-02-02-01.wav
processing ... archive/Actor_01/03-02-02-02-01-01-01.wav
processing ... archive/Actor_01/03-02-02-02-01-02-01.wav
processing ... archive/Actor_01/03-02-02-02-02-01-01.wav
processing ... archive/Actor_01/03-02-02-02-02-02-01.wav
processing ... archive/Actor_01/03-02-03-01-01-01-01.wav
processing ... archive/Actor_01/03-02-03-01-01-02-01.wav
processing ... archive/Actor_01/03-02-03-01-02-01-01.wav
processing ... archive/Actor_01/03-02-03-01-02-02-01.wav
processing ... archive/Actor_01/03-02-03-02-01-01-01.wav
processing ... archive/Actor_01